# Dislocation dynamics (2D)

Some python code to animate dislocation dynamics for teaching purposes. Needs to be tidied up, decribed and made more pythonic (and faster).

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
%matplotlib qt5

In [2]:
def initDD(n,D):
#  we use the rand function, which creates a 1D array of random
#  numbers between 0 and 1
    x_array = np.random.random(n)*D
    y_array = np.random.random(n)*D
    b_array = np.ones(n)
# assign b with equal numbers of +1 and -1
    b_array[:n//2]=1
    b_array[n//2:]=-1
    alive=np.ones(n, dtype=bool)
    return x_array,y_array,b_array,alive

def sumDD(n,D,rc,x_pos,y_pos,b_list,alive):
#  set force component to 0
    dx_list=[]
    dy_list=[]
    for x,y in zip(x_pos,y_pos):
        dx = x-x_pos
        dy = y-y_pos
        dx = dx - D*np.round(dx/D)
        dy = dy - D*np.round(dy/D)
        dx_list.append(dx)
        dy_list.append(dy)

        
    fx_array=np.zeros(n)
    for dis1 in range(n):
        fx=np.zeros(n)
        for dis2 in range(n):
            dx=dx_list[dis1][dis2]
            dy=dy_list[dis1][dis2]
            b1=b_list[dis1]
            b2=b_list[dis2]
            dsq = dx**2 + dy**2
            dist = np.sqrt(dsq)
            if np.logical_and(alive[dis1], alive[dis2]):
                if np.logical_and(dist<1.2,b1==-b2):
                    alive[dis1]=False
                    alive[dis2]=False
            if np.logical_and(alive[dis1], alive[dis2]):
                if (0.9 < dist <= rc):
                    fx[dis1] = fx[dis1] + b1*b2*dx*(dx**2-dy**2)/dsq**2
            fx_array=fx_array+fx

# calculate maximum value of force
    fmax=max(abs(fx_array))
    return fx_array,fmax,alive

#  input:  ndis = the number of dislocations
#          nsteps = number of time steps
#          dxmax = maximum move per time step
#
#  output: xi=initial x positions,x = final positions,y,b
#          fx = final forces on each dislocation
#          xdm = the maximum distance a dislocation has moved in the
#          calculation
def DD2D(ndis,nsteps,dxmax):
    xdm_list=[]
    dx_list=[]
    x_pos_list=[]
    y_pos_list=[]
    s_list=[]
    # set size of system (arbitrary)
    D = 100
    # set cutoff to be 1/2 the box length
    rc = D/2
    # initial positions
    x_pos,y_pos,b_list,alive = initDD(ndis,D)
    x_pos_list.append(x_pos.copy())
    y_pos_list.append(y_pos.copy())
    # store initial position
    xi = x_pos.copy()
    # start the time steps
    for step in range(nsteps):
        fx_array,fmax,alive = sumDD(ndis,D,rc,x_pos,y_pos,b_list,alive)
        dt = dxmax/fmax
        x_pos=x_pos+fx_array*dt
        escr=np.where(x_pos>D)
        escl=np.where(x_pos<0)
        x_pos[escr]=x_pos[escr]-D
        x_pos[escl]=x_pos[escl]+D
        xd = x_pos - xi#change in position during run
        xdm=np.max(abs(xd))# find maximum movement
        xdm_list.append(xdm.copy())
        dx_list.append(np.sum(np.abs(fx_array*dt)))
        x_pos_list.append(x_pos.copy())
        y_pos_list.append(y_pos.copy())
        s_list.append(alive)
        xd = xd - D*np.round(xd/D)# remove movement across periodic boundaries
    return xi,x_pos_list,y_pos_list,b_list,xdm_list,dx_list,s_list



In [3]:
def sumDD_climb(n,D,rc,x_pos,y_pos,b_list,alive):
#  set force component to 0
    dx_list=[]
    dy_list=[]
    for x,y in zip(x_pos,y_pos):
        dx = x-x_pos
        dy = y-y_pos
        dx = dx - D*np.round(dx/D)
        dy = dy - D*np.round(dy/D)
        dx_list.append(dx)
        dy_list.append(dy)
     
    fx_array=np.zeros(n)
    fy_array=np.zeros(n)
    for dis1 in range(n):
        fx=np.zeros(n)
        fy=np.zeros(n)
        for dis2 in range(n):
            dx=dx_list[dis1][dis2]
            dy=dy_list[dis1][dis2]
            b1=b_list[dis1]
            b2=b_list[dis2]
            dsq = dx**2 + dy**2
            dist = np.sqrt(dsq)
            if np.logical_and(alive[dis1], alive[dis2]):
                if np.logical_and(dist<1.5,b1==-b2):
                    alive[dis1]=False
                    alive[dis2]=False
            if np.logical_and(alive[dis1], alive[dis2]): # check for anihilation
                if (0.9 < dist <= rc):
                    fx[dis1] = fx[dis1] + b1*b2*dx*(dx**2-dy**2)/dsq**2
                    fy[dis1] = fy[dis1] - b1*b2*dy*(3*dx**2+dy**2)/dsq**2
            fx_array=fx_array+fx
            fy_array=fy_array+fy
    # calculate maximum value of force
    fmax=max([max(abs(fx_array)),max(abs(fy_array))])
    return fx_array, fy_array ,fmax,alive

#  input:  ndis = the number of dislocations
#          nsteps = number of time steps
#          dxmax = maximum move per time step
#
#  output: xi=initial x positions,x = final positions,y,b
#          fx = final forces on each dislocation
#          xdm = the maximum distance a dislocation has moved in the
#          calculation
def DD2D_climb(ndis,nsteps,dxmax):
    xdm_list=[]
    ydm_list=[]
    dy_list=[]
    dx_list=[]
    x_pos_list=[]
    y_pos_list=[]
    s_list=[]
    # set size of system (arbitrary)
    D = 100
    # set cutoff to be 1/2 the box length
    rc = D/2
    # initial positions
    x_pos,y_pos,b_list,alive = initDD(ndis,D)
    x_pos_list.append(x_pos.copy())
    y_pos_list.append(y_pos.copy())
    s_list.append(alive.copy())
    # store initial position
    xi = x_pos.copy()
    yi = y_pos.copy()
    # start the time steps
    for step in range(nsteps):
        fx_array,fy_array,fmax,alive = sumDD_climb(ndis,D,rc,x_pos,y_pos,b_list,alive)
        dt = dxmax/fmax
        x_pos=x_pos+fx_array*dt
        y_pos=y_pos+fy_array*dt*0.1
        escr=np.where(x_pos>D)
        escl=np.where(x_pos<0)
        x_pos[escr]=x_pos[escr]-D
        x_pos[escl]=x_pos[escl]+D
        escr=np.where(y_pos>D)
        escl=np.where(y_pos<0)
        y_pos[escr]=y_pos[escr]-D
        y_pos[escl]=y_pos[escl]+D
        xd = x_pos - xi#change in position during run
        yd= x_pos - yi #where is yi?
        #continue from here
        xdm=np.max(abs(xd))# find maximum x movement
        xdm_list.append(xdm.copy())
        ydm=np.max(abs(yd))
        ydm_list.append(ydm.copy())
        dx_list.append(np.sum(np.abs(fx_array*dt)))
        dy_list.append(np.sum(np.abs(fy_array*dt)))
        x_pos_list.append(x_pos.copy())
        y_pos_list.append(y_pos.copy())
        s_list.append(alive.copy())
        xd = xd - D*np.round(xd/D)# remove movement across periodic boundaries
        yd = yd - D*np.round(yd/D)
    return xi,x_pos_list,y_pos_list,b_list,xdm_list,dx_list,s_list

In [4]:
def plot_dis(x_pos,y_pos,n):
    mbot = ((-7, 0), (0, 0), (0, -8), (0, 0), (7, 0))
    mtop = ((-7, 0), (0, 0), (0, 8), (0, 0), (7, 0))
    fig, ax = plt.subplots()
    scat = ax.scatter(x_pos[:n//2],y_pos[:n//2],c='red',s=200,cmap='viridis',marker=mtop)
    scat2 = ax.scatter(x_pos[n//2:],y_pos[n//2:],c='blue',s=200,cmap='viridis',marker=mbot)

In [5]:
mbot = ((-7, 0), (0, 0), (0, -8), (0, 0), (7, 0))
mtop = ((-7, 0), (0, 0), (0, 8), (0, 0), (7, 0))

## Test functions (no climb)

Initiate dislocation array:

In [6]:
x_pos_list=[]
y_pos_list=[]
for n in range(10):
    D =100
    rc=D/2
    n_dis=20
    x_pos,y_pos,b_list,alive = initDD(n_dis,D)
    #plot_dis(x_pos,y_pos,n_dis)
    x_pos_list.append(x_pos.copy())
    y_pos_list.append(y_pos.copy())

### Create position lists with climb

In [18]:
n_dis=20
n_steps=400
max_dx=0.3
xi,x_pos_list_glide,y_pos_list_glide,b_list,xdm,f_res,s_list_glide = DD2D(n_dis,n_steps,max_dx)

### Create positions with climb

In [15]:
n_dis=20
n_steps=500
max_dx=0.7
xi,x_pos_list_climb,y_pos_list_climb,b_list,xdm,f_res,s_list_glide = DD2D_climb(n_dis,n_steps,max_dx)

##  Animate

Choose glide only or climb:

In [19]:
x_pos_list = x_pos_list_glide
x_pos_list = x_pos_list_glide
s_list = s_list_glide

#x_pos_list = x_pos_list_climb
#x_pos_list = x_pos_list_glide
#s_list = s_list_glide

In [20]:
mbot = ((-7, 0), (0, 0), (0, -8), (0, 0), (7, 0))
mtop = ((-7, 0), (0, 0), (0, 8), (0, 0), (7, 0))

D=100
n=n_dis
fig = plt.figure(figsize=(10,10))
ax=plt.axes(xlim=(0,D),ylim=(0,D))
line_up, = ax.plot([],[],marker=mtop,ms=20,lw=0,mew=2)
line_down, = ax.plot([],[],marker=mbot,ms=20,lw=0,mew=2)
time_step =ax.text(0,101,'')


def init():
    line_up.set_data([],[]) 
    line_down.set_data([],[])
    time_step.set_text('Time step = 0')             
    return line_up, line_down, time_step
    
def animate(i):
    x_up=x_pos_list[i][:n//2]
    x_up=x_up[np.where(s_list[i][:n//2])]
    y_up=y_pos_list[i][:n//2]
    y_up=y_up[np.where(s_list[i][:n//2])]
    x_down=x_pos_list[i][n//2:]
    x_down=x_down[np.where(s_list[i][n//2:])]
    y_down=y_pos_list[i][n//2:]
    y_down=y_down[np.where(s_list[i][n//2:])]
    line_up.set_data(x_up,y_up) 
    line_down.set_data(x_down,y_down)# update the data
    time_step.set_text('Time step = {:}'.format(i)) 
    return line_up, line_down,

ani = animation.FuncAnimation(fig, animate, init_func=init ,frames=n_steps,
                              interval=1,blit=False)

### Save the animation as a gif

In [17]:
ani.save('glide.gif',writer='imagemagick',fps=30)